# Document Understanding Solution - Question Answering


---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/introduction_to_applying_machine_learning|identify_key_insights_from_textual_document|document_question_answering.ipynb)

---


Question Answering is useful when you want to query a large amount of text for specific information. Maybe you're interested in extracting the
date a certain event happened. You can construct a question (or query) in natural language to retrive this information: e.g. 'When did Company X release Product Y?". Similar to extractive summarization we saw in the last notebook, Question Answering returns a verbatim slice of the
text as the answer. It won't generate new words to answer the question. In this notebook, we demonstrate three use cases of Questions and Answering:

1. How to directly deploy a pretrained Transformer-based extractive question answering model to perform inference.
2. How to fine-tune a pre-trained Transformer model on a custom dataset, and then run inference on the fine-tuned model.
3. How to run [SageMaker Automatic Model Tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html) (a hyperparameter optimization procedure) to find the best model compared with the model fine-tuned in point 2. The performance of the optimal model and model fine-tuned in point 2 is evaluated on a hold-out test data. 

**Note**: When running this notebook on SageMaker Studio, you should make
sure the `PyTorch 1.10 Python 3.8 CPU Optimized` image/kernel is used. When
running this notebook on SageMaker Notebook Instance, you should make
sure the 'sagemaker-soln' kernel is used.

## 1. Set Up

Before executing the notebook, there are some initial steps required for setup. This notebook requires latest version of sagemaker and ipywidgets.

In [ ]:
!pip install -U sagemaker ipywidgets

We start by importing a variety of packages that are used throughout
the notebook. One of the most important packages is the Amazon SageMaker
Python SDK (i.e. `import sagemaker`). We also import modules from our own
custom (and editable) package that can be found at `../package`.

In [ ]:
import sys
import boto3
import json
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker.local import LocalSession
import config
import datetime

IAM_ROLE = sagemaker.get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()
DEFAULT_BUCKET = sess.default_bucket()

Up next, we define the current folder and create a SageMaker client (from
`boto3`). We can use the SageMaker client to call SageMaker APIs
directly, as an alternative to using the Amazon SageMaker SDK. We use
it at the end of the notebook to delete certain resources that are
created in this notebook.

In [ ]:
current_folder = config.get_current_folder(globals())
sagemaker_client = boto3.client("sagemaker")

## 2. Run inference on the pre-trained extractive question answering model

Our question answering system needs a machine learning model. In this
section, we deploy a model to an Amazon SageMaker Endpoint and then
invoke the endpoint from the notebook. We use a pre-trained model from
the [transformers](https://huggingface.co/transformers/) library instead
of training a model from scratch, specifically the BERT Large model that
has been pre-trained on the SQuAD dataset.

We use the unique solution prefix to name the model and endpoint.

In [ ]:
import uuid

unique_hash = str(uuid.uuid4())[:6]
endpoint_name = f"{config.SOLUTION_PREFIX}-{unique_hash}-question-answering-endpoint"

### 2.1. Deploy an endpoint

Up next, we need to define the Amazon SageMaker Model which references
the source code and the specifies which container to use. 

Our pre-trained model is Extractive Question Answering model [bert-large-uncased-whole-word-masking-finetuned-squad](https://huggingface.co/bert-large-uncased-whole-word-masking-finetuned-squad) built on a Transformer model from Hugging Face. It takes two strings as inputs: the first string is a question and the second string is the context or any text you want to use to find the answer of the question, and it returns a sub-string from the context as an answer to the question.

We use the PyTorchModel from the Amazon SageMaker Python SDK. Using PyTorchModel and setting the `framework_version` argument, means that our deployed model runs inside a container that has PyTorch pre-installed (i.e., downloading the model on the fly). Other requirements can be installed by defining a `requirements.txt` file at the specified source_dir location. We use the `entry_point` argument to reference the code (within `source_dir`) that should be run for model inference: functions called `model_fn`, `input_fn`, `predict_fn` and `output_fn` are expected to be defined. And lastly, you can pass `model_data` from a training job, but we are going to load the pre-trained model in the source code running on the endpoint. We still
need to provide `model_data`, so we pass an empty archive.

In [ ]:
model = PyTorchModel(
    model_data=f"{config.SOURCE_S3_PATH}/artifacts/models/empty.tar.gz",
    entry_point="entry_point.py",
    source_dir="containers/question_answering",
    role=IAM_ROLE,
    framework_version="1.5.0",
    py_version="py3",
    code_location="s3://" + DEFAULT_BUCKET + "/code",
    env={
        "MODEL_ASSETS_S3_BUCKET": config.SOURCE_S3_BUCKET,
        "MODEL_ASSETS_S3_PREFIX": f"{config.SOURCE_S3_PREFIX}/artifacts/models/question_answering/",
        "MMS_DEFAULT_RESPONSE_TIMEOUT": "3000",
    },
)

Using this Amazon SageMaker Model, we can deploy a HTTPS endpoint on a
dedicated instance. We choose to deploy the endpoint on a single
ml.p3.2xlarge instance (or ml.g4dn.2xlarge if unavailable in this
region). Our question answering model is transfomer that
benefits from GPU optimization, and a ml.p3.2xlarge has a high
performance NVIDIA V100 GPU that can reduce inference latency on each
request. You can expect this deployment step to take around 5 minutes.
After approximately 15 dashes, you can expect to see an exclamation mark
which indicates a successful deployment.

In [ ]:
import time
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor = model.deploy(
    endpoint_name=endpoint_name,
    instance_type=config.HOSTING_INSTANCE_TYPE,
    initial_instance_count=1,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

time.sleep(10)

When you're trying to update the model for development purposes, but
experiencing issues because the model/endpoint-config/endpoint already
exists, you can delete the existing model/endpoint-config/endpoint by
uncommenting and running the following commands:

In [ ]:
# sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
# sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_name)

When calling our new endpoint from the notebook, we use a Amazon
SageMaker SDK
[`Predictor`](https://sagemaker.readthedocs.io/en/stable/predictors.html).
A `Predictor` is used to send data to an endpoint (as part of a request),
and interpret the response. Our `model.deploy` command returned a
`Predictor` but, by default, it sends and receive numpy arrays. Our
endpoint expects to receive (and also sends) JSON formatted objects, so
we modify the `Predictor` to use JSON instead of the PyTorch endpoint
default of numpy arrays. JSON is used here because it is a standard
endpoint format and the endpoint response can contain nested data
structures.

### 2.2. Example input sentences for inference & Query endpoint

With our model successfully deployed and our predictor configured, we can
try out the question answering model out on example inputs. All we need
to do is construct a dictionary object with two keys. `context` is the
text that we wish to retrieve information from. `question` is the natural
language query which specifices what information we're interested in
extracting. We call `predict` on our predictor and we should get a
response from the endpoint that contains the most likely answers.

In [ ]:
data = {"question": "what is my name?", "context": "my name is thom"}
response = predictor.predict(data=data)

We have the responce and we can print out the most likely answers that
has been extracted from the text above. You'll see each answer has a
confidence score used for ranking (but this score shouldn't be
interpreted as a true probability). In addition to the verbatim answer,
you also get the start and end character indexes of the answer from the
original context.

In [ ]:
print(response["answers"])

You can try more examples above, but note that this model has been
pretrained on the SQuAD dataset. You may need to fine-tune this model
with your own question answering data to obtain better results.

### 2.3. Clean up the endpoint

When you've finished with the summarization endpoint (and associated
endpoint-config), make sure that you delete it to avoid accidental
charges.

In [ ]:
# Delete the SageMaker endpoint and the attached resources
predictor.delete_model()
predictor.delete_endpoint()

## 3. Finetune the pre-trained model on a custom dataset

Previously, we saw how to run inference on a pre-trained extractive qusetion answering model. Next, we discuss how a model can be finetuned to a custom dataset. 

The Text Embedding model can be fine-tuned on any extractive question 
answering dataset in the same way the model available for inference has been 
fine-tuned on the SQuAD2.0 dataset.
The model available for fine-tuning attaches an answer extracting layer to the Text Embedding model
and initializes the layer parameters to random values. The fine-tuning step fine-tunes 
all the model parameters to minimize prediction error on the input data and returns the fine-tuned model.
The model returned by fine-tuning can be further deployed for inference. Below are the instructions 
for how the training data should be formatted for input to the model. 

- **Input:**  A directory containing a 'data.csv' file.
    - The first column of the 'data.csv' should have a question.
    - The second column should have the corresponding context.
    - The third column should have the integer character starting position for the answer in the context.
    - The fourth column should have the integer character ending position for the answer in the context.
- **Output:** A trained model that can be deployed for inference. 
 
Below is an example of 'data.csv' file showing values in its first four columns. Note that the file should not have any header.

|   |  |  |   |
|---|---|---|---|
|In what country is Normandy located?|	The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.|	159|	165
|...   | ... |...  | ... |
 

SQuAD2.0 dataset is downloaded from 
[Dataset Homepage](https://rajpurkar.github.io/SQuAD-explorer/). 
[CC BY-SA 4.0 License](https://creativecommons.org/licenses/by-sa/4.0/legalcode).

### 3.1. Retrieve JumpStart Training artifacts
Here, for the selected model, we retrieve the training docker container, the training algorithm source, the pre-trained model, and a python dictionary of the training hyper-parameters that the algorithm accepts with their default values. Note that the `model_version`="*" fetches the latest model. Also, we do need to specify the `training_instance_type` to fetch `train_image_uri`.

You can continue with the default model id, or can choose a different model from the dropdown generated upon running the next cell.

In [ ]:
model_id = "huggingface-eqa-bert-base-uncased"

In [ ]:
# download JumpStart model_manifest file.
boto3.client("s3").download_file(
    f"jumpstart-cache-prod-{aws_region}", "models_manifest.json", "models_manifest.json"
)
with open("models_manifest.json", "rb") as json_file:
    model_list = json.load(json_file)

# filter-out all the Text Classification models from the manifest list.
eqa_models_all_versions, eqa_models = [
    model["model_id"] for model in model_list if "-eqa-" in model["model_id"]
], []
[eqa_models.append(model) for model in eqa_models_all_versions if model not in eqa_models]

print(f"All the other available extractive question answering models are as below.\n")
for each in eqa_models:
    print(f"{each}")

In [ ]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters

model_version = "*"
training_instance_type = config.TRAINING_INSTANCE_TYPE

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=model_id,
    model_version=model_version,
    image_scope="training",
    instance_type=training_instance_type,
)
# Retrieve the training script
train_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="training"
)
# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="training"
)

### 3.2. Set Training parameters

Now that we are done with all the setup that is needed, we are ready to fine-tune our extractive question answering model. To begin, let us create a [``sageMaker.estimator.Estimator``](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) object. This estimator launchs the training job. 

There are two kinds of parameters that need to be set for training. 

The first one are the parameters for the training job. These include: (i) Training data path. This is S3 folder in which the input data is stored, (ii) Output path: This the s3 folder in which the training output is stored. (iii) Training instance type: This indicates the type of machine on which to run the training. Typically, we use GPU instances for these training. We defined the training instance type above to fetch the correct train_image_uri. 

The second set of parameters are algorithm specific training hyper-parameters.

In [ ]:
# Sample training data is available in this bucket
training_data_bucket = f"jumpstart-cache-prod-{aws_region}"
# For a quick demonstration of training we have created a random subset of SQuAD-v2 dataset.
# For complete QNLI dataset replace "SQuAD-v2-tiny" with "SQuAD-v2" in the line below.
training_data_prefix = "training-datasets/SQuAD-v2-tiny/"

training_dataset_s3_path = f"s3://{training_data_bucket}/{training_data_prefix}"

output_bucket = DEFAULT_BUCKET
output_prefix = "EQA"

s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"

For algorithm specific hyper-parameters, we start by fetching python dictionary of the training hyper-parameters that the algorithm accepts with their default values. This can then be overridden to custom values.

In [ ]:
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)

# [Optional] Override default hyperparameters with custom values
hyperparameters["batch-size"] = "16"
print(hyperparameters)

### 3.3. Download, preprocess, and upload the training data

In [ ]:
!aws s3 cp --recursive $training_dataset_s3_path data/squad2

In [ ]:
import pandas as pd

data = pd.read_csv("data/squad2/data.csv", header=None)

View the first five observations of the training data

In [ ]:
data.head(5)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.15, random_state=42)

In [ ]:
train_data.to_csv("data/squad2/split_train.csv", header=False, index=False)

In [ ]:
import os

prefix = "EQA"
boto3.Session().resource("s3").Bucket(DEFAULT_BUCKET).Object(
    os.path.join(prefix, "train/data.csv")
).upload_file("data/squad2/split_train.csv")

Process the text data to make them ready for inference. In particular, the question with multiple answers are formulated as multiple rows in above `data`. For an example, one question with three answers will yield three rows in the `data`. Each row corresponds to one answer and has the same question content. In evalution, we combine those rows that correspond to the same question. As a result, each input in test examples has an unique question content and its corresponded ground truth answers can be muliple. For each test example for inference, which includes one context and question, the model outputs a predicted answer. Next. we compare the predicted answer with each of the ground truth answers and use the best comparision result for model performance on that example. Details are shown in the inference section as below.

In [ ]:
unique_test_examples = {}

for idx, row in test_data.iterrows():
    if row[0] not in unique_test_examples:
        unique_test_examples[row[0]] = {
            "context": row[1],
            "answer": [row[1][row[2] : row[3]]],
        }
    else:
        assert row[1] == unique_test_examples[row[0]]["context"]
        unique_test_examples[row[0]]["answer"].append(row[1][row[2] : row[3]])

In [ ]:
test_examples = []
ground_truth = []
for key in unique_test_examples:
    test_examples.append([key, unique_test_examples[key]["context"]])
    ground_truth.append(unique_test_examples[key]["answer"])

### 3.4. Fine-tuning without hyperparameter optimization

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base
from sagemaker.tuner import HyperparameterTuner

from sagemaker import get_execution_role

role = get_execution_role()

training_job_name = training_job_name = f"{config.SOLUTION_PREFIX}-eqa-finetune"

# Create SageMaker Estimator instance
eqa_estimator = Estimator(
    role=role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    tags=[{"Key": config.TAG_KEY, "Value": config.SOLUTION_PREFIX}],
    base_job_name=training_job_name,
    debugger_hook_config=False,
)


training_data_path_updated = f"s3://{DEFAULT_BUCKET}/{prefix}/train"
# Launch a SageMaker Training job by passing s3 path of the training data
eqa_estimator.fit({"training": training_data_path_updated}, logs=True)

## 3.5. Deploy & run Inference on the fine-tuned model

A trained model does nothing on its own. We now want to use the model to perform inference. For this example, that means predicting the answer of an input sentence including a context and question. We follow the same steps as in `2. Run inference on the pre-trained extractive question answering model`. We start by retrieving the jumpstart artifacts for deploying an endpoint. However, instead of base_predictor, we  deploy the `eqa_estimator` that we fine-tuned.

In [ ]:
inference_instance_type = config.HOSTING_INSTANCE_TYPE

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)

endpoint_name_finetune = (
    f"{config.SOLUTION_PREFIX}eqa-finetune-{datetime.datetime.now().strftime('%Y-%m-%d-%H%M%S')}"
)

# Use the estimator from the previous step to deploy to a SageMaker endpoint
finetuned_predictor = eqa_estimator.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name_finetune,
)

time.sleep(10)

In [ ]:
newline, bold, unbold = "\n", "\033[1m", "\033[0m"


def query_endpoint(encoded_text, predictor):
    response = predictor.predict(
        encoded_text,
        {"ContentType": "application/list-text", "Accept": "application/json;verbose"},
    )
    return response


def parse_response(query_response):
    model_predictions = json.loads(query_response)
    answer = (model_predictions["answer"],)
    return answer

In [ ]:
predictions = []
for question_context in test_examples:
    query_response = query_endpoint(
        json.dumps(question_context).encode("utf-8"), finetuned_predictor
    )
    answer = parse_response(query_response)
    predictions.append(answer[0])

In [ ]:
# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))


def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()

    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)

    common_tokens = set(pred_tokens) & set(truth_tokens)

    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0

    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)

    return 2 * (prec * rec) / (prec + rec)

In [ ]:
import numpy as np

em_score, f1_score = [], []

for prediction, gold_answers in zip(predictions, ground_truth):
    em_score.append(max((compute_exact_match(prediction, answer)) for answer in gold_answers))
    f1_score.append(max((compute_f1(prediction, answer)) for answer in gold_answers))

print(f"Average Exact Matching score: {np.mean(em_score)}")
print(f"Average F1 score: {np.mean(f1_score)}")

In [ ]:
result = {
    "Average Exact Matching score": [np.mean(em_score)],
    "Average F1 Score": [np.mean(f1_score)],
}

In [ ]:
result = pd.DataFrame.from_dict(result, orient="index", columns=["No HPO"])

In [ ]:
result

## 4. Finetune the pre-trained model on a custom dataset with automatic model tuning (AMT)

Amazon SageMaker automatic model tuning, also known as hyperparameter tuning, finds the best version of a model by running many training jobs on your dataset using the algorithm and ranges of hyperparameters that you specify. It then chooses the hyperparameter values that result in a model that performs the best, as measured by a metric that you choose. We use a [HyperparameterTuner](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html) object to interact with Amazon SageMaker hyperparameter tuning APIs.

In [ ]:
from sagemaker.tuner import (
    ContinuousParameter,
    IntegerParameter,
    CategoricalParameter,
    HyperparameterTuner,
)


# Define objective metric per framework, based on which the best model is selected.
metric_definitions_per_model = {
    "huggingface": {
        "metrics": [{"Name": "val_loss", "Regex": "'eval_loss': ([0-9]+(.|e\-)[0-9]+),?"}],
        "type": "Minimize",
    }
}

# You can select from the hyperparameters supported by the model, and configure ranges of values to be searched for training the optimal model.(https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-define-ranges.html)
hyperparameter_ranges = {
    "adam-learning-rate": ContinuousParameter(0.00001, 0.01, scaling_type="Logarithmic"),
    "epochs": IntegerParameter(3, 10),
    "train-only-top-layer": CategoricalParameter(["True", "False"]),
}

# Increase the total number of training jobs run by AMT, for increased accuracy (and training time).
max_jobs = 6
# Change parallel training jobs run by AMT to reduce total training time, constrained by your account limits.
# if max_jobs=max_parallel_jobs then Bayesian search turns to Random.
max_parallel_jobs = 3

### 4.1. Fine-tuning with hyperparameter optimization

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.tuner import HyperparameterTuner

tuning_job_name = f"{config.SOLUTION_PREFIX}-eqa-hpo"

# Create SageMaker Estimator instance
eqa_estimator = Estimator(
    role=role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    tags=[{"Key": config.TAG_KEY, "Value": config.SOLUTION_PREFIX}],
    base_job_name=tuning_job_name,
    debugger_hook_config=False,
)

metric_definitions = next(
    value for key, value in metric_definitions_per_model.items() if model_id.startswith(key)
)

hp_tuner = HyperparameterTuner(
    eqa_estimator,
    metric_definitions["metrics"][0]["Name"],
    hyperparameter_ranges,
    metric_definitions["metrics"],
    max_jobs=max_jobs,
    max_parallel_jobs=max_parallel_jobs,
    objective_type=metric_definitions["type"],
    base_tuning_job_name=training_job_name,
)

# Launch a SageMaker Tuning job to search for the best hyperparameters
hp_tuner.fit({"training": training_data_path_updated})

### 4.2. Deploy & run Inference on the fine-tuned model

In [ ]:
# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)

endpoint_name_hpo = f"{config.SOLUTION_PREFIX}-eqa-hpo-endpoint"

# Use the estimator from the previous step to deploy to a SageMaker endpoint
finetuned_predictor_hpo = hp_tuner.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name_hpo,
)

time.sleep(10)

In [ ]:
predictions_hpo = []
for question_context in test_examples:
    query_response = query_endpoint(
        json.dumps(question_context).encode("utf-8"), finetuned_predictor_hpo
    )
    answer = parse_response(query_response)
    predictions_hpo.append(answer[0])

In [ ]:
import numpy as np

em_score_hpo, f1_score_hpo = [], []

for prediction, gold_answers in zip(predictions_hpo, ground_truth):
    em_score_hpo.append(max((compute_exact_match(prediction, answer)) for answer in gold_answers))
    f1_score_hpo.append(max((compute_f1(prediction, answer)) for answer in gold_answers))

print(f"Average Exact Matching score: {np.mean(em_score_hpo)}")
print(f"Average F1 score: {np.mean(f1_score_hpo)}")

In [ ]:
result_hpo = {
    "Average Exact Matching score": [np.mean(em_score_hpo)],
    "Average F1 Score": [np.mean(f1_score_hpo)],
}

In [ ]:
result_hpo = pd.DataFrame.from_dict(result_hpo, orient="index", columns=["With HPO"])

In [ ]:
pd.concat([result, result_hpo], axis=1)

We can see results with hyperparameter optimization shows better performance on the hold-out test data.

## 4.3. Clean Up the endpoint

When you've finished with the summarization endpoint (and associated
endpoint-config), make sure that you delete it to avoid accidental
charges.

In [ ]:
# Delete the SageMaker endpoint and the attached resources
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()

finetuned_predictor_hpo.delete_model()
finetuned_predictor_hpo.delete_endpoint()

## Next Stage

We've just looked at how you can query document for specific information.
Up next we look at a technique that can be used to extract the key
entities from a document, called Entity Recognition.

[Click here to continue with Name Entity Recognition.](./4_entity_recognition.ipynb)

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/introduction_to_applying_machine_learning|identify_key_insights_from_textual_document|document_question_answering.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/introduction_to_applying_machine_learning|identify_key_insights_from_textual_document|document_question_answering.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/introduction_to_applying_machine_learning|identify_key_insights_from_textual_document|document_question_answering.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/introduction_to_applying_machine_learning|identify_key_insights_from_textual_document|document_question_answering.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/introduction_to_applying_machine_learning|identify_key_insights_from_textual_document|document_question_answering.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/introduction_to_applying_machine_learning|identify_key_insights_from_textual_document|document_question_answering.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/introduction_to_applying_machine_learning|identify_key_insights_from_textual_document|document_question_answering.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/introduction_to_applying_machine_learning|identify_key_insights_from_textual_document|document_question_answering.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/introduction_to_applying_machine_learning|identify_key_insights_from_textual_document|document_question_answering.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/introduction_to_applying_machine_learning|identify_key_insights_from_textual_document|document_question_answering.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/introduction_to_applying_machine_learning|identify_key_insights_from_textual_document|document_question_answering.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/introduction_to_applying_machine_learning|identify_key_insights_from_textual_document|document_question_answering.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/introduction_to_applying_machine_learning|identify_key_insights_from_textual_document|document_question_answering.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/introduction_to_applying_machine_learning|identify_key_insights_from_textual_document|document_question_answering.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/introduction_to_applying_machine_learning|identify_key_insights_from_textual_document|document_question_answering.ipynb)
